In [ ]:
# A simple class to know about your cuda devices
import pycuda.driver as cuda
import pycuda.autoinit # Necessary for using its functions
cuda.init() # Necesarry for using its functions

class aboutCudaDevices():
    def __init__(self):
        pass
    
    def num_devices(self):
        """Return number of devices connected."""
        return cuda.Device.count()
    
    def devices(self):
        """Get info on all devices connected."""
        num = cuda.Device.count()
        print("%d device(s) found:"%num)
        for i in range(num):
            print(cuda.Device(i).name(), "(Id: %d)"%i)
            
    def mem_info(self):
        """Get available and total memory of all devices."""
        available, total = cuda.mem_get_info()
        print("Available: %.2f GB\nTotal:     %.2f GB"%(available/1e9, total/1e9))
        
    def attributes(self, device_id=0):
        """Get attributes of device with device Id = device_id"""
        return cuda.Device(device_id).get_attributes()
    
    def __repr__(self):
        """Class representation as number of devices connected and about them."""
        num = cuda.Device.count()
        string = ""
        string += ("%d device(s) found:\n"%num)
        for i in range(num):
            string += ( "    %d) %s (Id: %d)\n"%((i+1),cuda.Device(i).name(),i))
            string += ("          Memory: %.2f GB\n"%(cuda.Device(i).total_memory()/1e9))
        return string

# You can print output just by typing its name (__repr__):
aboutCudaDevices()
# 1 device(s) found:
#    1) Tesla K80 (Id: 0)
#          Memory: 12.00 GB

In [ ]:
# Returns the current GPU memory usage by 
# tensors in bytes for a given device
torch.cuda.memory_allocated()
# Returns the current GPU memory managed by the
# caching allocator in bytes for a given device
torch.cuda.memory_reserved()

# Releases all unoccupied cached memory currently held by
# the caching allocator so that those can be used in other
# GPU application and visible in nvidia-smi
torch.cuda.empty_cache()

In [4]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import transformers
import torch.utils.data as tdata
import torch.optim as optim

import tqdm
import os
import pycuda.driver as cuda
import re as re
#torch.cuda.is_available()
#torch.cuda.current_device()
#cuda.Device(0).name()
#torch.cuda.get_device_name(0)

from transformers import AutoModel, AutoTokenizer

In [187]:
print(os.listdir('dados/'))

train = pd.read_csv('dados/Hackathon_Base_Treino_comdep.csv')
test = pd.read_csv('dados/Hackathon_Base_Teste.csv', header=0, index_col=0)

['Catalogo Categorias e Sub Categorias.csv', 'Dados Gerais Datasets.txt', 'dados_tokenV001.csv', 'dados_tokenV002.csv', 'dados_tokenV003.csv', 'dotzTranslate-cf435230f08e.json', 'Hackathon_Base_Teste.csv', 'Hackathon_Base_Treino_comdep.csv']


In [188]:
class PreProcessing:
    def __init__(self):
        self.dados = []
        
        def putCharBtw(x,letra,sep):
            rex = re.compile('[0-9]'+letra)
            if rex.search(x)==None:
                return x
            else:
                inicio = (rex.search(x).start())+1
                newString = x[:inicio]+sep+x[inicio:]
                return newString
        self.putCharBtw = putCharBtw
    
    def putData(self, data, campos):
        for campo in campos:
            newcolumn = campo+'ajuste'
            data[newcolumn] = data[[campo]].apply(lambda x: x.str.upper()).copy()
            data[newcolumn] = data[newcolumn].str.replace(' C/',' COM ').copy()
            data[newcolumn] = data[newcolumn].str.replace('\.C/',' COM ', regex=True).copy()
            data[newcolumn] = data[newcolumn].str.replace('-C/',' COM ', regex=True).copy()
            
            data[newcolumn] = data[newcolumn].str.replace(' S/',' SEM ').copy()
            data[newcolumn] = data[newcolumn].str.replace('\.S/',' SEM ', regex=True).copy()
            data[newcolumn] = data[newcolumn].str.replace('-S/',' SEM ', regex=True).copy()
            
            data[newcolumn] = data[newcolumn].str.replace(' P/',' PARA ').copy()
            data[newcolumn] = data[newcolumn].str.replace('\.P/',' PARA ', regex=True).copy()
            data[newcolumn] = data[newcolumn].str.replace('-P/',' PARA ', regex=True).copy()
            
            data[newcolumn] = data[newcolumn].str.replace('/',' / ', regex=True).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['ML',' ']).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['KG',' ']).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['G',' ']).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['CM',' ']).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['L',' ']).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['MM',' ']).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['MT',' ']).copy()
            
            data[newcolumn] = data[newcolumn].str.replace('-',' - ', regex=True).copy()

        return print('finish')

In [189]:
preprocess = PreProcessing()
preprocess.putData(train,['DESCRIÇÃO PARCEIRO'])
preprocess.putData(test,['DESCRIÇÃO PARCEIRO'])

finish
finish


In [190]:
pd.set_option('display.min_rows', 30)
train[train['DESCRIÇÃO PARCEIROajuste'].str.contains('[0-9] L')]

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESCRIÇÃO PARCEIROajuste
8,"SORVETE NOBRELLI 1,5L PT FLOCOS",POTE TRADICIONAL,SORVETE,CONGELADOS DOCES,"SORVETE NOBRELLI 1,5 L PT FLOCOS"
10,T.IMEDIA EXCELL KIT GTS CO.ELSEVE 7.1 LOUR.ACZ.,PERMANENTE LOURO,COLORAÇÃO FEMININA,CUIDADOS COM O CABELO,T.IMEDIA EXCELL KIT GTS CO.ELSEVE 7.1 LOUR.ACZ.
15,SUCO SUVALAN 100% UVA 1L,REGULAR,SUCO PRONTO/NECTAR,BEBIDAS NÃO ALCOÓLICAS,SUCO SUVALAN 100% UVA 1 L
20,"COMBO MONTAIN DEW 1,5L-PET LV2PG1",LIMÃO,REFRIGERANTES,BEBIDAS NÃO ALCOÓLICAS,"COMBO MONTAIN DEW 1,5 L - PET LV2PG1"
25,LAVA ROUPA LIQ BRILHANTE 1L,INTERMEDIÁRIO,LAVA ROUPA LÍQUIDO,CUIDADOS COM A ROUPA,LAVA ROUPA LIQ BRILHANTE 1 L
46,GARRAFA TERMICA EASY MOR 1L,GARRAFA TÉRMICA,UTENSÍLIOS PARA CAFÉ,UTILIDADES DE COZINHA,GARRAFA TERMICA EASY MOR 1 L
61,VINHO NAC MONTE REALE BAG 3L TINTO SUAVE,VINHO NACIONAL,VINHO,BEBIDAS ALCOÓLICAS,VINHO NAC MONTE REALE BAG 3 L TINTO SUAVE
67,CONDICIONADOR 1L BIOEXTRATUS TUTANO,TRADICIONAL,CONDICIONADOR,CUIDADOS COM O CABELO,CONDICIONADOR 1 L BIOEXTRATUS TUTANO
74,"SORVETE JUNDIA MAX CHOC BRANCO POTE 1,5LT",POTE TRADICIONAL,SORVETE,CONGELADOS DOCES,"SORVETE JUNDIA MAX CHOC BRANCO POTE 1,5 LT"
82,LIXEIRA KREA PLAST C/PEDAL 5L OTIN19,LIXEIRA,LIXEIRA,LAVANDERIA,LIXEIRA KREA PLAST COM PEDAL 5 L OTIN19


In [192]:
from sklearn.preprocessing import LabelEncoder
cats_encoder = LabelEncoder()
train['encoded_CATEGORIA'] = cats_encoder.fit_transform(train['CATEGORIA'])
train['encoded_DEPARTAMENTO'] = cats_encoder.fit_transform(train['DEPARTAMENTO'])

In [193]:
train

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESCRIÇÃO PARCEIROajuste,encoded_CATEGORIA,encoded_DEPARTAMENTO
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,"PASTA INT VITAPOWER 1,005 KG AMEND / SHOT",85,39
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS COM 1,124,11
2,AGUA MIN SCHIN S/GAS 500ML,SEM GÁS,ÁGUA MINERAL,BEBIDAS NÃO ALCOÓLICAS,AGUA MIN SCHIN SEM GAS 500 ML,327,3
3,FITA DUPLA FACE C/SUPORTE SCOTCH,FITA ADESIVA,PAPELARIA,PAPELARIA,FITA DUPLA FACE COM SUPORTE SCOTCH,230,42
4,MASSA PIZZA ROMANHA OREGANO PCT 160G,PIZZA REGULAR,MASSA FRESCA,MASSAS FRESCAS,MASSA PIZZA ROMANHA OREGANO PCT 160 G,197,38
5,BISC LANCH BAUDUCCO RECHEADINHO CHOCOLATE 104G,LANCHINHO,BISCOITO,MERCEARIA DOCE,BISC LANCH BAUDUCCO RECHEADINHO CHOCOLATE 104 G,44,39
6,VINHO FRANCES CHEVALIER CHABLIS BCO 750ML,VINHO FRANCÊS,VINHO,BEBIDAS ALCOÓLICAS,VINHO FRANCES CHEVALIER CHABLIS BCO 750 ML,322,2
7,DES ROLL REXONA INVISIBLE 50ML,ROLL ON COM PERFUME,DESODORANTE MASCULINO,CUIDADOS COM O CORPO,DES ROLL REXONA INVISIBLE 50 ML,103,16
8,"SORVETE NOBRELLI 1,5L PT FLOCOS",POTE TRADICIONAL,SORVETE,CONGELADOS DOCES,"SORVETE NOBRELLI 1,5 L PT FLOCOS",286,7
9,NOZ MOSCADA KONSABOR 10G,NOZ MOSCADA,CONDIMENTO NATURAL,MERCEARIA SALGADA,NOZ MOSCADA KONSABOR 10 G,80,40


In [194]:
#pd.set_option('display.max_rows', 100)
#pd.set_option('max_rows', 100)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)
pd.set_option('display.expand_frame_repr', True)

train[train['DESCRIÇÃO PARCEIRO'].str.contains('[0-9]')]#.contains('^f')
#train[['descricao_ParceiroAjuste']].apply(lambda x: x.str.upper())
#train['descricao_ParceiroAjuste'] = train['descricao_ParceiroAjuste'].str.replace('(?=[\S])ml', ' ml', regex=True).copy()

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESCRIÇÃO PARCEIROajuste,encoded_CATEGORIA,encoded_DEPARTAMENTO
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,"PASTA INT VITAPOWER 1,005 KG AMEND / SHOT",85,39
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS COM 1,124,11
2,AGUA MIN SCHIN S/GAS 500ML,SEM GÁS,ÁGUA MINERAL,BEBIDAS NÃO ALCOÓLICAS,AGUA MIN SCHIN SEM GAS 500 ML,327,3
4,MASSA PIZZA ROMANHA OREGANO PCT 160G,PIZZA REGULAR,MASSA FRESCA,MASSAS FRESCAS,MASSA PIZZA ROMANHA OREGANO PCT 160 G,197,38
5,BISC LANCH BAUDUCCO RECHEADINHO CHOCOLATE 104G,LANCHINHO,BISCOITO,MERCEARIA DOCE,BISC LANCH BAUDUCCO RECHEADINHO CHOCOLATE 104 G,44,39
6,VINHO FRANCES CHEVALIER CHABLIS BCO 750ML,VINHO FRANCÊS,VINHO,BEBIDAS ALCOÓLICAS,VINHO FRANCES CHEVALIER CHABLIS BCO 750 ML,322,2
7,DES ROLL REXONA INVISIBLE 50ML,ROLL ON COM PERFUME,DESODORANTE MASCULINO,CUIDADOS COM O CORPO,DES ROLL REXONA INVISIBLE 50 ML,103,16
8,"SORVETE NOBRELLI 1,5L PT FLOCOS",POTE TRADICIONAL,SORVETE,CONGELADOS DOCES,"SORVETE NOBRELLI 1,5 L PT FLOCOS",286,7
9,NOZ MOSCADA KONSABOR 10G,NOZ MOSCADA,CONDIMENTO NATURAL,MERCEARIA SALGADA,NOZ MOSCADA KONSABOR 10 G,80,40
10,T.IMEDIA EXCELL KIT GTS CO.ELSEVE 7.1 LOUR.ACZ.,PERMANENTE LOURO,COLORAÇÃO FEMININA,CUIDADOS COM O CABELO,T.IMEDIA EXCELL KIT GTS CO.ELSEVE 7.1 LOUR.ACZ.,75,15


In [195]:
pd.get_option("display.max_rows")

100

In [196]:
len(train['DEPARTAMENTO'].value_counts())

53

In [197]:
device = torch.device("cuda:0")
#roberta_weights = 'roberta-base'
# Using the community model
# BERT Base
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
model = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased').to(device)

In [199]:
tokenized = [torch.tensor(tokenizer
                          .encode(x)).unsqueeze(0).to(device) for x in tqdm.notebook.tqdm(train['DESCRIÇÃO PARCEIROajuste'])]

In [200]:
embeddings = []

model.eval()
with torch.no_grad():
  for x in tqdm.notebook.tqdm(tokenized):
    
    embeddings.append(model(x)[1].cpu().numpy())

In [201]:
embeddings_numpy = np.array(embeddings).squeeze()

In [203]:
DFToken = pd.DataFrame(embeddings_numpy)
dadosTrain = train.join(DFToken)
#.to_csv('data/dados_tokenV001.csv')
dadosTrain.to_csv('dados/dados_tokenV004.csv', sep=';')

In [172]:
dadosTrain.drop(["DESCRIÇÃO PARCEIRO",'SUB-CATEGORIA', 'CATEGORIA', 'DEPARTAMENTO', 'DESCRIÇÃO PARCEIROajuste','encoded_CATEGORIA', 'encoded_DEPARTAMENTO'], axis=1)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.112895,-0.105799,0.026719,0.056843,-0.000890,0.135331,0.994743,0.017883,0.013704,0.069881,...,0.050450,-0.167555,-0.070584,0.568091,-0.087728,-0.024671,-0.040177,0.149683,-0.082165,-0.071840
1,0.052492,-0.148830,0.011367,0.020572,-0.011979,0.063353,0.999208,0.001538,-0.031455,0.155275,...,0.026674,-0.221637,-0.016349,0.077433,0.023592,-0.105960,0.023350,0.245084,0.052775,-0.110047
2,0.077723,-0.071447,-0.073816,0.069799,-0.103397,0.090352,0.865802,0.073956,-0.065923,0.075312,...,0.036042,-0.103226,-0.083071,0.917473,-0.070158,-0.108429,-0.025564,0.045246,-0.201900,-0.024803
3,-0.047794,-0.087092,0.117663,0.130383,-0.045416,0.083600,0.999400,0.167612,-0.043209,0.053562,...,0.082982,-0.166233,-0.056189,0.107748,0.041448,-0.078493,-0.102621,0.244331,-0.037084,-0.044428
4,0.157124,-0.087438,-0.073976,0.130672,-0.039189,0.127748,0.996870,-0.037604,0.008412,0.096224,...,0.075219,-0.287787,-0.114509,0.441673,-0.148186,-0.085901,-0.081996,0.133701,-0.060037,-0.078798
5,0.054224,-0.242601,-0.181527,0.058180,-0.010039,-0.006554,0.998151,0.089014,-0.002312,0.114314,...,-0.027558,-0.256654,-0.081340,0.368430,-0.112593,-0.244104,0.038009,0.225887,-0.061173,-0.085735
6,-0.164106,-0.114741,-0.087051,0.005116,-0.116768,-0.092608,-0.049333,0.072172,-0.018428,0.049596,...,-0.123924,-0.182251,-0.069627,0.964018,-0.141869,-0.136718,-0.068594,0.204761,-0.219864,-0.053376
7,-0.068271,-0.071111,-0.063279,0.054264,-0.134093,0.013623,0.990722,-0.009677,-0.075457,0.089425,...,-0.046902,-0.136340,-0.118766,0.596669,-0.090468,-0.064236,-0.110180,0.067421,-0.163547,0.013956
8,0.044820,-0.254434,-0.013773,0.062843,-0.030734,0.149964,0.995551,0.216361,0.031199,0.052082,...,0.042140,-0.127291,0.011934,0.602488,-0.153503,-0.155948,0.062369,0.174325,-0.054966,-0.042268
9,0.159846,-0.192621,-0.183020,0.102984,-0.013323,0.013148,0.998802,0.004693,0.075079,0.073503,...,0.001094,-0.302510,-0.092539,0.307837,-0.114018,-0.200264,-0.075323,0.248549,-0.035166,-0.030126


In [174]:
trainFinal = dadosTrain.drop(["DESCRIÇÃO PARCEIRO",'SUB-CATEGORIA', 'CATEGORIA', 'DEPARTAMENTO', 'DESCRIÇÃO PARCEIROajuste','encoded_CATEGORIA', 'encoded_DEPARTAMENTO'], axis=1).copy()

In [175]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

Xtrain, Xval, ytrain, yval = train_test_split(trainFinal, train['encoded_CATEGORIA'] , train_size=0.5, random_state=0)

print(Xtrain.shape, Xval.shape, ytrain.shape, yval.shape)

(11004, 768) (11005, 768) (11004,) (11005,)


In [176]:
from sklearn.metrics import f1_score


In [ ]:
predictions = np.zeros((Xval.shape[0], 332))

for class_ in tqdm.notebook.tqdm(sorted(train['encoded_CATEGORIA'].unique())):

  mask_train = ytrain==class_
  mask_val = yval==class_

  if mask_train.sum() + mask_val.sum() < 10:
    continue

  ytrain_ = mask_train.astype(int)
  yval_ = mask_val.astype(int)

  model = LogisticRegression(class_weight='auto')
  model.fit(Xtrain, ytrain_)
  #como retorna a probabilidade para 0 e a probabilidade para 1 pega-se somente a segunda coluna
  p = model.predict_proba(Xval)[:,1]
    
  predictions[:, class_] = p

  threshold = np.percentile(p, 99)

  p_cut = (p > threshold).astype(int)

  #print("Class = {} | Num exemplos positivos | train = {} | val = {} | F1 = {} | p-avg = {}\n".format(class_, ytrain_.sum(), yval_.sum(), f1_score(yval_, p_cut), p.mean()))
  #break

C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

In [179]:
predictions = np.zeros((Xval.shape[0], 332))

for class_ in tqdm.notebook.tqdm(sorted(train['encoded_CATEGORIA'].unique())):

  mask_train = ytrain==class_
  mask_val = yval==class_

  if mask_train.sum() + mask_val.sum() < 10:
    continue

  ytrain_ = mask_train.astype(int)
  yval_ = mask_val.astype(int)

  model = LogisticRegression(class_weight='auto')
  model.fit(Xtrain, ytrain_)
  #como retorna a probabilidade para 0 e a probabilidade para 1 pega-se somente a segunda coluna
  p = model.predict_proba(Xval)[:,1]
    
  predictions[:, class_] = p

  threshold = np.percentile(p, 99)

  p_cut = (p > threshold).astype(int)

  #print("Class = {} | Num exemplos positivos | train = {} | val = {} | F1 = {} | p-avg = {}\n".format(class_, ytrain_.sum(), yval_.sum(), f1_score(yval_, p_cut), p.mean()))
  #break

C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

...

C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MANAGER\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

In [181]:
predictions

array([[6.76539456e-04, 2.01097149e-03, 1.05098139e-02, ...,
        8.14047698e-05, 7.72242880e-06, 5.97803018e-05],
       [6.95191978e-04, 2.37351880e-03, 4.16301606e-03, ...,
        3.26330890e-04, 1.73317714e-04, 5.79114586e-04],
       [8.17161101e-04, 1.00971131e-04, 5.53829317e-04, ...,
        2.39317321e-04, 3.73923009e-04, 1.56482611e-04],
       ...,
       [7.55588225e-05, 1.58973528e-03, 1.02820004e-03, ...,
        2.11990702e-04, 2.86323608e-04, 1.19704102e-04],
       [4.79238644e-04, 3.14912438e-03, 3.29929783e-04, ...,
        7.34669710e-03, 1.74508118e-03, 3.48029370e-04],
       [1.44306336e-04, 4.41444170e-05, 6.57868213e-05, ...,
        9.65052541e-04, 2.37516868e-03, 5.45635666e-03]])

In [183]:
p_argmax = predictions.argmax(axis=1)
f1_score(yval, p_argmax, average='micro')

0.5391185824625171

In [184]:
from scipy.stats import rankdata

p_top_rank = np.zeros(predictions.shape)
for class_ in range(predictions.shape[1]):
  p_top_rank[:,class_] = rankdata(predictions[:, class_])
p_top_rank = p_top_rank.argmax(axis=1)
f1_score(yval,p_top_rank, average='micro')

0.41181281235801903

# PRE-PROCESSING TESTE

In [7]:
class PreProcessing:
    def __init__(self):
        self.dados = []
        
        def putCharBtw(x,letra,sep):
            rex = re.compile('[0-9]'+letra)
            if rex.search(x)==None:
                return x
            else:
                inicio = (rex.search(x).start())+1
                newString = x[:inicio]+sep+x[inicio:]
                return newString
        self.putCharBtw = putCharBtw
    
    def putData(self, data, campos):
        for campo in campos:
            newcolumn = campo+'ajuste'
            data[newcolumn] = data[[campo]].apply(lambda x: x.str.upper()).copy()
            data[newcolumn] = data[newcolumn].str.replace(' C/',' COM ').copy()
            data[newcolumn] = data[newcolumn].str.replace('\.C/',' COM ', regex=True).copy()
            data[newcolumn] = data[newcolumn].str.replace('-C/',' COM ', regex=True).copy()
            
            data[newcolumn] = data[newcolumn].str.replace(' S/',' SEM ').copy()
            data[newcolumn] = data[newcolumn].str.replace('\.S/',' SEM ', regex=True).copy()
            data[newcolumn] = data[newcolumn].str.replace('-S/',' SEM ', regex=True).copy()
            
            data[newcolumn] = data[newcolumn].str.replace(' P/',' PARA ').copy()
            data[newcolumn] = data[newcolumn].str.replace('\.P/',' PARA ', regex=True).copy()
            data[newcolumn] = data[newcolumn].str.replace('-P/',' PARA ', regex=True).copy()
            
            data[newcolumn] = data[newcolumn].str.replace('/',' / ', regex=True).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['ML',' ']).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['KG',' ']).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['G',' ']).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['CM',' ']).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['L',' ']).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['MM',' ']).copy()
            data[newcolumn] = data[newcolumn].apply(self.putCharBtw, args=['MT',' ']).copy()
            
            data[newcolumn] = data[newcolumn].str.replace('-',' - ', regex=True).copy()

        return print('finish')

In [16]:
test = pd.read_csv('dados/Hackathon_Base_Teste.csv', index_col=0,sep=',', header=0)
test = test.reset_index(drop=True).copy()
preprocess = PreProcessing()
preprocess.putData(test,['DESCRIÇÃO PARCEIRO'])

finish


In [17]:
test

,DESCRIÇÃO PARCEIRO,DESCRIÇÃO PARCEIROajuste
0,,
1,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA COM ALHO 500 G
2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA COM 17 DR / PR / BC R952
3,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO,MIST PREP LIQ CHA VDE LINEA 56 ML - CX LIMAO
4,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO
5,MEIA PIZZA PEPPERONI UN,MEIA PIZZA PEPPERONI UN
...,...,...
9429,BISC IMP ITAL GRISBI DOCE CAPPUCCINO CX 150G,BISC IMP ITAL GRISBI DOCE CAPPUCCINO CX 150 G
9430,VINHO ARGENTINO SANTA ANA BCO SECO 700ML,VINHO ARGENTINO SANTA ANA BCO SECO 700 ML
9431,ALGODAO BOLA ISABABY 50G .,ALGODAO BOLA ISABABY 50 G .


In [11]:
device = torch.device("cuda:0")
#roberta_weights = 'roberta-base'
# Using the community model
# BERT Base
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
model = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased').to(device)
tokenized = [torch.tensor(tokenizer
                          .encode(x)).unsqueeze(0).to(device) for x in tqdm.notebook.tqdm(test['DESCRIÇÃO PARCEIROajuste'])]

In [12]:
embeddings = []

model.eval()
with torch.no_grad():
  for x in tqdm.notebook.tqdm(tokenized):
    
    embeddings.append(model(x)[1].cpu().numpy())

In [32]:
embeddings_numpy = np.array(embeddings).squeeze()
DFToken = pd.DataFrame(embeddings_numpy)
dadosTest = test.join(DFToken)
#.to_csv('data/dados_tokenV001.csv')


In [35]:
dadosTest = test.reset_index(drop=True).copy()
dadosTest = dadosTest.join(DFToken)

In [37]:
dadosTest.to_csv('dados/test_tokenV004.csv', sep=';')